In [1]:
# Ignore SQLITE warnings related to Decimal numbers in the Chinook database
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

In [19]:
# Create an engine for the chinook.sqlite database
engine = create_engine("sqlite:///chinook.sqlite", echo=False)
#engine = create_engine("sqlite:///titanic.sqlite", echo=False)

In [20]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'tracks',
 'media_types',
 'playlists']

In [5]:
# Save a reference to the invoices table as `Invoices`
Invoices = Base.classes.invoices

In [6]:
# Create a database session object
session = Session(engine)

In [7]:
# List all of the countries found in the invoices table
session.query(Invoices.BillingCountry).group_by(Invoices.BillingCountry).all()

# Alternative Solution using `distinct`
# session.query(Invoices.BillingCountry).distinct().all()

[('Argentina',),
 ('Australia',),
 ('Austria',),
 ('Belgium',),
 ('Brazil',),
 ('Canada',),
 ('Chile',),
 ('Czech Republic',),
 ('Denmark',),
 ('Finland',),
 ('France',),
 ('Germany',),
 ('Hungary',),
 ('India',),
 ('Ireland',),
 ('Italy',),
 ('Netherlands',),
 ('Norway',),
 ('Poland',),
 ('Portugal',),
 ('Spain',),
 ('Sweden',),
 ('USA',),
 ('United Kingdom',)]

In [8]:
# Design a query that lists the invoices totals for each billing country 
# and sort the output in descending order.
session.query(Invoices.BillingCountry, func.sum(Invoices.Total)).\
    group_by(Invoices.BillingCountry).\
    order_by(func.sum(Invoices.Total).desc()).all()

[('USA', Decimal('523.06')),
 ('Canada', Decimal('303.96')),
 ('France', Decimal('195.10')),
 ('Brazil', Decimal('190.10')),
 ('Germany', Decimal('156.48')),
 ('United Kingdom', Decimal('112.86')),
 ('Czech Republic', Decimal('90.24')),
 ('Portugal', Decimal('77.24')),
 ('India', Decimal('75.26')),
 ('Chile', Decimal('46.62')),
 ('Ireland', Decimal('45.62')),
 ('Hungary', Decimal('45.62')),
 ('Austria', Decimal('42.62')),
 ('Finland', Decimal('41.62')),
 ('Netherlands', Decimal('40.62')),
 ('Norway', Decimal('39.62')),
 ('Sweden', Decimal('38.62')),
 ('Poland', Decimal('37.62')),
 ('Italy', Decimal('37.62')),
 ('Denmark', Decimal('37.62')),
 ('Australia', Decimal('37.62')),
 ('Argentina', Decimal('37.62')),
 ('Spain', Decimal('37.62')),
 ('Belgium', Decimal('37.62'))]

In [9]:
# Save a reference to the invoice_items table as `Items`
Items = Base.classes.invoice_items

In [10]:
# List all of the Billing Postal Codes for the USA.
results = session.query(Invoices.BillingPostalCode).\
    filter(Invoices.BillingCountry == 'USA').group_by(Invoices.BillingPostalCode).all()
results

[('10012-2612',),
 ('2113',),
 ('32801',),
 ('53703',),
 ('60611',),
 ('76110',),
 ('84102',),
 ('85719',),
 ('89503',),
 ('94040-111',),
 ('94043-1351',),
 ('95014',),
 ('98052-8300',)]

In [11]:
# Calculate the Item Totals (sum(UnitPrice * Quantity)) for the USA
session.query(func.sum(Items.UnitPrice * Items.Quantity)).\
    filter(Invoices.InvoiceId == Items.InvoiceId).\
    filter(Invoices.BillingCountry == 'USA').scalar()

Decimal('523.0600000000')

In [12]:
# Calculate the Item Totals `sum(UnitPrice * Quantity)` for each Billing Postal Code in the USA
# Sort the results in descending order by Total
session.query(Invoices.BillingPostalCode, func.sum(Items.UnitPrice * Items.Quantity)).\
    filter(Invoices.InvoiceId == Items.InvoiceId).\
    filter(Invoices.BillingCountry == 'USA').\
    group_by(Invoices.BillingPostalCode).\
    order_by(func.sum(Items.UnitPrice * Items.Quantity).desc()).all()

[('76110', Decimal('47.6200000000')),
 ('60611', Decimal('43.6200000000')),
 ('84102', Decimal('43.6200000000')),
 ('53703', Decimal('42.6200000000')),
 ('94040-111', Decimal('39.6200000000')),
 ('98052-8300', Decimal('39.6200000000')),
 ('32801', Decimal('39.6200000000')),
 ('95014', Decimal('38.6200000000')),
 ('94043-1351', Decimal('37.6200000000')),
 ('89503', Decimal('37.6200000000')),
 ('85719', Decimal('37.6200000000')),
 ('2113', Decimal('37.6200000000')),
 ('10012-2612', Decimal('37.6200000000'))]